In [1]:
import hashlib
import inspect
import types

In [2]:
def fdigest(filename):
    f = open(filename,mode='rb')
    m = hashlib.sha256(f.read())
    f.close()
    return m.hexdigest()

In [3]:
h = fdigest('Loads.ipynb')
h

'd5b691758babfd8d3ddd6ad3842c08c87c825361fd9eaf8ec904ac0a7faca53d'

In [4]:
def extend(new):
    name = new.__name__
    old = globals().get(name,None)
    if old is None:
        old = type(name,(object,),{})
    ##print name,'Old:',id(old),old
    ##print 'New:',id(new), new
    ok = ['__init__']
    for a,v in inspect.getmembers(new):
        if not a.startswith('_') or a in ok:
            ##print 'Type of:',a,'=',type(v)
            if type(v) is types.MethodType:
                v = types.MethodType(v.im_func,v.im_self,old)
            elif type(v) is property:
                v = property(v.fget,v.fset,v.fdel)
            elif type(v) is types.FunctionType:
                v = staticmethod(types.FunctionType(v.func_code,v.func_globals,v.func_name,v.func_defaults,v.func_closure))
                #v = staticmethod(v.func_code)
            setattr(old,a,v)
            ##print 'Set', a, 'as type', type(v)
    return old

In [5]:
class Foo(object):
    def one(self):
        return 1

In [6]:
id(Foo), Foo

(3042263556L, __main__.Foo)

In [7]:
@extend
class Foo:
    def two(self):
        return 2

In [8]:
@extend
class Foo:
    def __init__(s):
        s.__p = 13
    def three(self):
        return 3
    @property
    def p(s):
        return s,s.__p
    @p.setter
    def p(s,v):
        s.__p = v
    @classmethod
    def cm(c):
        return 'class',c
    @staticmethod
    def sm():
        return 77

In [9]:
f = Foo()

In [10]:
f.one()

1

In [11]:
f.two()

2

In [12]:
f.three()

3

In [13]:
f.p

(<__main__.Foo at 0xb5556eec>, 13)

In [14]:
f.p = 3686868

In [15]:
f.p

(<__main__.Foo at 0xb5556eec>, 3686868)

In [16]:
f.cm()

('class', <class __main__.Foo at 0xb55eeddc>)

In [20]:
f.sm()

77

In [18]:
class Zap(object):
    pass

In [19]:
def mc(*l):
    print 'mc:',l
def mc2(*l):
    print 'mc2:',l

@mc2
class Zipp(object):
    __metaclass__ = mc
    A = 1
    def z(s):
        return s

mc: ('Zipp', (<type 'object'>,), {'A': 1, '__module__': '__main__', 'z': <function z at 0xb556b064>, '__metaclass__': <function mc at 0xb5542e9c>})
mc2: (None,)
